In [1]:
print("hello")

hello


In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

# Initialize WebDriver
driver = webdriver.Chrome()

# Open the URL
driver.get("https://www.amazon.in/gp/bestsellers/kitchen/5689363031/")

# Loop to find the 'gridItemRoot' element 10 times and click each link
for i in range(10):
    try:
        # Find all grid items (multiple instances of the same element)
        grid_items = driver.find_elements(By.ID, "gridItemRoot")
        
        # Check if there are enough grid items to click (ensure you have more than i items)
        if len(grid_items) > i:
            grid_item = grid_items[i]  # Get the (i+1)-th item
            
            # Find the link inside this gridItemRoot (you can adjust this if necessary)
            link = grid_item.find_element(By.TAG_NAME, "a")
            
            # Get the URL of the link (optional, for debugging)
            link_url = link.get_attribute("href")
            print(f"Opening link {i+1}: {link_url}")
            
            # Open the link by clicking on it
            link.click()
            
            # Wait for the new page to load (adjust time based on page load speed)
            time.sleep(.00001)
            
            # Go back to the previous page
            driver.back()
            
            # Wait a moment before continuing to the next link
            time.sleep(0.00001)
        else:
            print(f"Not enough grid items to process, exiting at iteration {i+1}")
            break  # Exit if there aren't enough items left

    except Exception as e:
        print(f"Error occurred at iteration {i+1}: {e}")
        break  # Exit if an error occurs (e.g., element not found)

# After completing 10 iterations, quit the driver
driver.quit()

Opening link 1: https://www.amazon.in/Wakefit-Engineered-Platform-Storage-Finish_Brown/dp/B08Q8DJRP1/ref=zg_bs_g_5689363031_d_sccl_1/259-9914403-6277001?psc=1
Opening link 2: https://www.amazon.in/Honey-Soft-Foam-Mattress-Children/dp/B09Z6CZFPQ/ref=zg_bs_g_5689363031_d_sccl_2/259-9914403-6277001?psc=1
Not enough grid items to process, exiting at iteration 3


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def fetch_tracking_details(driver, awbid, loginurl):
    driver.get(loginurl)

    try:
        # Select AWB option and enter AWB number
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "(//a[normalize-space()='AWB'])[1]"))).click()
        driver.find_element(By.XPATH, "(//input[@placeholder='Enter your AWB number'])[1]").send_keys(awbid)
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "(//button[normalize-space()='Track Order'])[1]"))).click()

        # Wait for the tracking details to load
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "(//button[normalize-space()='Tracking Details'])[1]")))

        details_sections = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'flex w-full p-4')]")))
        if details_sections:
            print("\nTracking Details:")
            for detail in details_sections:
                try:
                    # Check and extract the status
                    status_elements = detail.find_elements(By.XPATH, ".//div[contains(@class, 'font-semibold')]")
                    status = status_elements[0].text if status_elements else "Unknown status"
                    
                    # Extract additional details
                    details_text = detail.find_elements(By.XPATH, ".//div[@class='text-[12px] font-normal md:w-[250px]']")
                    details = "\n".join([elem.text for elem in details_text]) if details_text else "No additional details found."

                    print(f"Status: {status}\nDetails: {details}\n")
                except Exception as inner_e:
                    print(f"Could not find specific detail elements: {inner_e}")
        else:
            print("No additional tracking details found.")
    except Exception as e:
        print(f"An error occurred while fetching tracking information: {e}")

    finally:
        driver.quit()

if __name__ == "__main__":
    awbid = "11311514500355"
    loginurl = "https://www.delhivery.com/"
    driver = initialize_driver()
    fetch_tracking_details(driver, awbid, loginurl)



Tracking Details:
Status: PICKED UP
Details: 
There might be some delay due to holiday closures. - Bangalore



In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Initialize the WebDriver
def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

# Function to fetch product variations from the given URL
def fetch_product_variations(driver, product_url):
    # Open the product URL
    driver.get(product_url)

    try:
        # Wait for the product title to load
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'productTitle')))

        # Fetch and print the product title
        product_title = driver.find_element(By.ID, 'productTitle').text
        print(f"Product Title: {product_title}\n")

        # Find all variation sections (e.g., Color, Size)
        variations = driver.find_elements(By.XPATH, "//div[contains(@id, 'variation_')]")
        for variation in variations:
            # Get the name of the variation (e.g., Color, Size)
            variation_name = variation.find_element(By.XPATH, ".//label").text.strip()
            print(f"Variation: {variation_name}")

            # Check for the currently selected option
            selected_option = variation.find_elements(By.XPATH, ".//span[@class='selection']")
            if selected_option:
                selected_option_text = selected_option[0].text.strip()
                print(f" - Selected: {selected_option_text}")

            # Get all available options for this variation
            options = variation.find_elements(By.XPATH, ".//li[contains(@class, 'swatchAvailable') or contains(@class, 'swatchSelect')]")
            for option in options:
                # Clean up the option text by removing unnecessary prefixes
                option_value = option.get_attribute("title").replace("Click to select ", "").strip()
                print(f" - {option_value}")

            print("")  # For better readability in the output

    except Exception as e:
        # Handle any errors that occur during the process
        print(f"An error occurred while fetching product variations: {e}")

    finally:
        # Close the browser
        driver.quit()

# Main execution block
if __name__ == "__main__":
    # URL of the Amazon product page
    product_url = "https://www.amazon.in/dp/B0B2PH6CDC"
    # Initialize the WebDriver
    driver = initialize_driver()
    # Fetch and print the product variations
    fetch_product_variations(driver, product_url)


Product Title: Wakefit Bed | King (78 X 72) Engineered Wood Bed with Storage, 1 Year Warranty | - Petra - Columbian Walnut

Variation: Size:
 - Selected: King
 - King
 - Queen

Variation: Style Name:



In [47]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import time

# Initialize the WebDriver
def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

# Scroll the element into view and click
def scroll_into_view_and_click(driver, element):
    driver.execute_script("arguments[0].scrollIntoView(true);", element)
    time.sleep(1)  # Wait for any animations to complete
    element.click()

# Function to fetch product variations and their prices
def fetch_product_variations(driver, product_url):
    driver.get(product_url)

    try:
        # Wait for the product title to load
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'productTitle')))
        
        # Fetch and print the product title
        product_title = driver.find_element(By.ID, 'productTitle').text
        print(f"Product Title: {product_title}\n")

        # Find all variation sections (e.g., Color, Size, Material)
        variations = driver.find_elements(By.XPATH, "//div[contains(@id, 'variation_') or contains(@class, 'twisterSlot')]")

        # Dictionary to store variation names and their options
        variation_options = {}

        for variation in variations:
            # Get the name of the variation (e.g., Color, Size)
            label_element = variation.find_elements(By.XPATH, ".//label | .//span[@class='a-dropdown-label']")
            if label_element:
                variation_name = label_element[0].text.strip()
                print(f"Processing variation: {variation_name}")

                # Check for swatch options (color, style, etc.)
                swatch_options = variation.find_elements(By.XPATH, ".//li[contains(@class, 'swatchAvailable') or contains(@class, 'swatchSelect')]")
                if swatch_options:
                    variation_options[variation_name] = swatch_options
                else:
                    # Check for dropdown options if no swatch options are found
                    dropdowns = variation.find_elements(By.XPATH, ".//select")
                    if dropdowns:
                        for dropdown in dropdowns:
                            select = Select(dropdown)
                            options = [option.text for option in select.options if option.text]
                            variation_options[variation_name] = options

                    # Check for checkboxes (additional options, if any)
                    checkbox_options = variation.find_elements(By.XPATH, ".//input[@type='checkbox']")
                    if checkbox_options:
                        variation_options[variation_name] = checkbox_options

                # If no variations found, set an empty list (in case of unsupported variations)
                if variation_name not in variation_options:
                    variation_options[variation_name] = []

                print(f"Found variation: {variation_name} with {len(variation_options[variation_name])} options")

        # If no variations found
        if not variation_options:
            print("No variations available.")
            return

        # Recursive function to handle nested selections
        def iterate_variations(variations, index, selections):
            if index == len(variations):
                # Apply the selections made and fetch product info
                for selection in selections:
                    if isinstance(selection, tuple):  # Handle Select object for dropdowns
                        select, option_text = selection
                        scroll_into_view_and_click(driver, select._el)  # Scroll to the dropdown
                        select.select_by_visible_text(option_text)
                    elif selection.tag_name == "input" and selection.get_attribute("type") == "checkbox":
                        if not selection.is_selected():
                            scroll_into_view_and_click(driver, selection)
                    else:
                        scroll_into_view_and_click(driver, selection)
                    time.sleep(2)

                # Fetch the updated product title and price
                updated_product_title = driver.find_element(By.ID, 'productTitle').text
                try:
                    price = driver.find_element(By.XPATH, "//div[@id='corePriceDisplay_desktop_feature_div']//span[@class='a-price-whole']").text
                except:
                    price = "Price not available"

                # Print the combination and the price
                combination = ", ".join([option_text if isinstance(selection, tuple) else selection.get_attribute('title') for selection, option_text in selections])
                print(f"Selected Combination: {combination}")
                print(f"Updated Product Title: {updated_product_title}")
                print(f"Price: ₹{price}\n")

                return

            # Iterate over the options for the current variation
            variation_name, options = list(variations.items())[index]
            for option in options:
                if isinstance(option, tuple):
                    iterate_variations(variations, index + 1, selections + [(option[0], option[1])])
                else:
                    iterate_variations(variations, index + 1, selections + [option])

        # Start the recursive iteration with the first variation
        iterate_variations(variation_options, 0, [])

    except Exception as e:
        print(f"An error occurred while fetching product variations: {e}")

    finally:
        driver.quit()

if __name__ == "__main__":
    # URL of the Amazon product page
    product_url = "https://www.amazon.in/dp/B0CZ3ZPD8B"
    # Initialize the WebDriver
    driver = initialize_driver()
    # Fetch and print the product variations
    fetch_product_variations(driver, product_url)


Product Title: boAt Airdopes 311 Pro Truly Wireless in Ear Ear Buds w/upto 50 HRS Playtime, Dual Mics with ENx™ Tech,50 ms Low-Latency Beast™ Mode,ASAP™ Charging, IPX4,IWP Tech & BT v5.3 earbuds tws (Active Black)

Processing variation: Colour:
Found variation: Colour: with 4 options
An error occurred while fetching product variations: cannot unpack non-iterable WebElement object


In [62]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Initialize the WebDriver
def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

# Function to fetch product variations and their prices
def fetch_product_variations(driver, product_url):
    driver.get(product_url)
    
    try:
        # Wait for the product title to load
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'productTitle')))
        
        # Fetch and print the product title
        product_title = driver.find_element(By.ID, 'productTitle').text
        print(f"Product Title: {product_title}\n")
        
        # Find all variation sections (e.g., Color, Size, Material)
        variations = driver.find_elements(By.XPATH, "//div[contains(@id, 'variation_') or contains(@class, 'twisterSlot')]")
        
        # Debug: Print the number of variations found
        print(f"Found {len(variations)} variations.")
        
        # If no variations are found, exit early
        if len(variations) == 0:
            print("No variations found on this page.")
            return
        
        # List to store all variation elements and their options
        variation_elements = []
        for variation in variations:
            try:
                # Get the variation name (like "Color", "Size", etc.)
                variation_name = variation.find_element(By.XPATH, ".//label").text.strip()
                # Find all the available options for this variation
                options = variation.find_elements(By.XPATH, ".//li[contains(@class, 'swatchAvailable') or contains(@class, 'swatchSelect')]")
                
                # If no swatches found, check if there are dropdowns (e.g., Size, Quantity)
                if not options:
                    dropdowns = variation.find_elements(By.XPATH, ".//select")
                    if dropdowns:
                        select = dropdowns[0]
                        options = select.find_elements(By.TAG_NAME, 'option')
                
                # Debug: Print the variation name and number of options
                print(f"Variation: {variation_name}, Options: {len(options)}")
                
                variation_elements.append((variation_name, options))
            except Exception as e:
                print(f"Could not process variation due to error: {e}")
        
        # Now process the variations and print their options
        if variation_elements:
            for variation_name, options in variation_elements:
                print(f"Variation: {variation_name}")
                for option in options:
                    print(f"  Option: {option.text}")

    except Exception as e:
        print(f"An error occurred while fetching product variations: {e}")
    finally:
        driver.quit()

if __name__ == "__main__":
    # URL of the Amazon product page
    product_url = "https://www.amazon.in/dp/B08Q8DJRP1"  # Change this to the desired product URL
    # Initialize the WebDriver
    driver = initialize_driver()
    # Fetch and print the product variations
    fetch_product_variations(driver, product_url)


Product Title: Wakefit Bed | Queen (78 X 60) Engineered Wood Bed with Storage, 1 Year Warranty | - Taurus - Columbian Walnut

Found 2 variations.
Variation: Size:, Options: 4
Variation: Style Name:, Options: 99
Variation: Size:
  Option: Single - 72"x36"
  Option: Single - 78"x36"
  Option: King
  Option: Queen
Variation: Style Name:
  Option: Algol Non Storage
  Option: Algol With Hydraulic Storage
  Option: Algol With Storage
  Option: Antho Upholstered With Storage - Velvet ...
  Option: Antho Upholstered With Storage - Velvet ...
  Option: Antho Upholstered With Storage - Velvet ...
  Option: Antho Upholstered With Storage - Velvet ...
  Option: Antho Upholstered With Storage - Velvet ...
  Option: Astra Non Storage
  Option: Astra With Hydraulic Storage
  Option: Astra With Storage
  Option: Caph Non Storage
  Option: Caph With Hydraulic Storage
  Option: Caph With Storage
  Option: Castor Non Storage
  Option: Castor With Hydraulic Storage
  Option: Castor With Storage
  Option: 

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import time

# Initialize the WebDriver
def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

# Function to fetch product variations and their prices
def fetch_product_variations(driver, product_url):
    driver.get(product_url)
    
    try:
        # Wait for the product title to load
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'productTitle')))
        
        # Fetch and print the product title
        product_title = driver.find_element(By.ID, 'productTitle').text
        print(f"Product Title: {product_title}\n")
        
        # Find all variation sections (e.g., Color, Size)
        variations = driver.find_elements(By.XPATH, "//div[contains(@id, 'variation_')]")
        for variation in variations:
            # Get the name of the variation (e.g., Color, Size)
            variation_name = variation.find_element(By.XPATH, ".//label").text.strip()
            print(f"Variation: {variation_name}")
            
            # Get all available options for this variation and click each one
            options = variation.find_elements(By.XPATH, ".//li[contains(@class, 'swatchAvailable') or contains(@class, 'swatchSelect')]")
            for option in options:
                option.click()
                time.sleep(2)  # Adding a short wait to ensure the page updates
                
                # Fetch the updated product title
                updated_product_title = driver.find_element(By.ID, 'productTitle').text
                
                # Fetch the updated price
                try:
                    price = driver.find_element(By.XPATH, "//div[@id='corePriceDisplay_desktop_feature_div']//span[@class='a-price-whole']").text
                except:
                    price = "Price not available"
                
                print(f"Selected Option: {option.get_attribute('title')}")
                print(f"Updated Product Title: {updated_product_title}")
                print(f"Price: ₹{price}\n")
            
            # Check for dropdown variations
            dropdowns = variation.find_elements(By.XPATH, ".//select")
            for dropdown in dropdowns:
                select = Select(dropdown)  # Initialize Select object
                
                # Fetch all dropdown options
                dropdown_options = select.options
                for dropdown_option in dropdown_options:
                    select.select_by_visible_text(dropdown_option.text)
                    time.sleep(2)  # Adding a short wait to ensure the page updates
                    
                    # Fetch the updated product title
                    updated_product_title = driver.find_element(By.ID, 'productTitle').text
                    
                    # Fetch the updated price
                    try:
                        price = driver.find_element(By.XPATH, "//div[@id='corePriceDisplay_desktop_feature_div']//span[@class='a-price-whole']").text
                    except:
                        price = "Price not available"
                    
                    print(f"Selected Option: {dropdown_option.text}")
                    print(f"Updated Product Title: {updated_product_title}")
                    print(f"Price: ₹{price}\n")
                    
            print("")  # For better readability in the output

    except Exception as e:
        print(f"An error occurred while fetching product variations: {e}")

    finally:
        driver.quit()

if __name__ == "__main__":
    # URL of the Amazon product page
    product_url = "https://www.amazon.in/dp/B08Q8DJRP1"
    # Initialize the WebDriver
    driver = initialize_driver()
    # Fetch and print the product variations
    fetch_product_variations(driver, product_url)


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import time

# Initialize the WebDriver
def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

# Function to fetch product variations and their prices
def fetch_product_variations(driver, product_url):
    driver.get(product_url)

    try:
        # Wait for the product title to load
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'productTitle')))

        # Fetch and print the product title
        product_title = driver.find_element(By.ID, 'productTitle').text
        print(f"Product Title: {product_title}\n")

        # Find all variation sections
        variations = driver.find_elements(By.XPATH, "//div[contains(@id, 'variation_')]")
        for variation in variations:
            # Get the name of the variation (e.g., Color, Size)
            variation_name = variation.find_element(By.XPATH, ".//label").text.strip()
            print(f"Variation: {variation_name}")

            # Get all available options for this variation and click each one
            options = variation.find_elements(By.XPATH, ".//li[contains(@class, 'swatchAvailable') or contains(@class, 'swatchSelect')]")
            for option in options:
                option.click()
                time.sleep(2)  # Adding a short wait to ensure the page updates

                # Fetch the updated product title
                updated_product_title = driver.find_element(By.ID, 'productTitle').text

                # Fetch the updated price
                try:
                    price = driver.find_element(By.XPATH, "//div[@id='corePriceDisplay_desktop_feature_div']//span[@class='a-price-whole']").text
                except:
                    price = "Price not available"

                print(f"Selected Option: {option.get_attribute('title')}")
                print(f"Updated Product Title: {updated_product_title}")
                print(f"Price: ₹{price}\n")

            # Check for dropdown variations
            dropdowns = variation.find_elements(By.XPATH, ".//select")
            for dropdown in dropdowns:
                select = Select(dropdown)  # Initialize Select object

                # Fetch all dropdown options
                dropdown_options = select.options
                for dropdown_option in dropdown_options:
                    select.select_by_visible_text(dropdown_option.text)
                    time.sleep(2)  # Adding a short wait to ensure the page updates

                    # Fetch the updated product title
                    updated_product_title = driver.find_element(By.ID, 'productTitle').text

                    # Fetch the updated price
                    try:
                        price = driver.find_element(By.XPATH, "//div[@id='corePriceDisplay_desktop_feature_div']//span[@class='a-price-whole']").text
                    except:
                        price = "Price not available"

                    print(f"Selected Option: {dropdown_option.text}")
                    print(f"Updated Product Title: {updated_product_title}")
                    print(f"Price: ₹{price}\n")

            print("")  # For better readability in the output

    except Exception as e:
        print(f"An error occurred while fetching product variations: {e}")

    finally:
        driver.quit()

if __name__ == "__main__":
    # URL of the Amazon product page
    product_url = "https://www.amazon.in/dp/B08Q8DJRP1"
    # Initialize the WebDriver
    driver = initialize_driver()
    # Fetch and print the product variations
    fetch_product_variations(driver, product_url)


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import time

# Initialize the WebDriver
def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

# Scroll the element into view and click
def scroll_into_view_and_click(driver, element):
    driver.execute_script("arguments[0].scrollIntoView(true);", element)
    time.sleep(1)  # Wait for any animations to complete
    element.click()

# Function to fetch product variations and their prices
def fetch_product_variations(driver, product_url):
    driver.get(product_url)

    try:
        # Wait for the product title to load
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'productTitle')))

        # Fetch and print the product title
        product_title = driver.find_element(By.ID, 'productTitle').text
        print(f"Product Title: {product_title}\n")

        # Find all variation sections
        variations = driver.find_elements(By.XPATH, "//div[contains(@id, 'variation_')]")
        variation_options = []

        # Collect all variation elements and their options
        for variation in variations:
            variation_name = variation.find_element(By.XPATH, ".//label").text.strip()
            print(f"Collecting options for Variation: {variation_name}")

            options = variation.find_elements(By.XPATH, ".//li[contains(@class, 'swatchAvailable') or contains(@class, 'swatchSelect')]")
            dropdowns = variation.find_elements(By.XPATH, ".//select")

            if options:
                variation_options.append(options)
            elif dropdowns:
                select = Select(dropdowns[0])
                dropdown_options = select.options
                variation_options.append(dropdown_options)

        # Recursive function to handle nested selections
        def iterate_variations(index, selections):
            if index == len(variation_options):
                # Apply the selections made and fetch product info
                for selection in selections:
                    if isinstance(selection, webdriver.remote.webelement.WebElement):
                        scroll_into_view_and_click(driver, selection)
                    else:
                        select = Select(selection[0])
                        select.select_by_visible_text(selection[1])
                    time.sleep(2)

                # Fetch the updated product title and price
                updated_product_title = driver.find_element(By.ID, 'productTitle').text
                try:
                    price = driver.find_element(By.XPATH, "//div[@id='corePriceDisplay_desktop_feature_div']//span[@class='a-price-whole']").text
                except:
                    price = "Price not available"

                combination = ", ".join([selection.text if isinstance(selection, webdriver.remote.webelement.WebElement) else selection[1] for selection in selections])
                print(f"Selected Combination: {combination}")
                print(f"Updated Product Title: {updated_product_title}")
                print(f"Price: ₹{price}\n")

                return

            for option in variation_options[index]:
                if isinstance(option, webdriver.remote.webelement.WebElement):
                    iterate_variations(index + 1, selections + [option])
                else:
                    iterate_variations(index + 1, selections + [(dropdowns[0], option.text)])

        # Start the recursive iteration with the first variation
        iterate_variations(0, [])

    except Exception as e:
        print(f"An error occurred while fetching product variations: {e}")

    finally:
        driver.quit()

if __name__ == "__main__":
    # URL of the Amazon product page
    product_url = "https://www.amazon.in/dp/B08698T6BY"
    driver = initialize_driver()
    fetch_product_variations(driver, product_url)


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import time

# Initialize the WebDriver
def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

# Scroll the element into view and click
def scroll_into_view_and_click(driver, element):
    driver.execute_script("arguments[0].scrollIntoView(true);", element)
    time.sleep(1)  # Wait for any animations to complete
    element.click()

# Function to fetch product variations and their prices
def fetch_product_variations(driver, product_url):
    driver.get(product_url)

    try:
        # Wait for the product title to load
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'productTitle')))

        # Fetch and print the product title
        product_title = driver.find_element(By.ID, 'productTitle').text
        print(f"Product Title: {product_title}\n")

        # Find all variation sections
        variations = driver.find_elements(By.XPATH, "//div[contains(@id, 'variation_')]")
        variation_options = []
        variation_names = []

        # Collect all variation elements and their options
        for index,variation in enumerate(variations):
            variation_name = variation.find_element(By.XPATH, ".//label").text.strip()
            variation_names.append(variation_name)
            print(f"Collecting options for Variation: {variation_name}")

            # Check if it is the first variation
            if index == 0:
                # First variation: consider all options (including swatchUnavailable)
                options = variation.find_elements(By.XPATH, ".//li[contains(@class, 'swatchAvailable') or contains(@class, 'swatchUnavailable') or contains(@class, 'swatchSelect')]")
            else:
                # For all other variations, only consider swatchAvailable and swatchSelect
                options = variation.find_elements(By.XPATH, ".//li[contains(@class, 'swatchAvailable') or contains(@class, 'swatchSelect')]")

            dropdowns = variation.find_elements(By.XPATH, ".//select")

            if options:
                variation_options.append(options)
            elif dropdowns:
                select = Select(dropdowns[0])
                dropdown_options = select.options
                variation_options.append(dropdown_options)

        # Recursive function to handle nested selections
        def iterate_variations(index, selections):
            if index == len(variation_options):
                # Apply the selections made and fetch product info
                for selection in selections:
                    if isinstance(selection, webdriver.remote.webelement.WebElement):
                        scroll_into_view_and_click(driver, selection)
                    else:
                        select = Select(selection[0])
                        select.select_by_visible_text(selection[1])
                    time.sleep(2)

                # Fetch the updated product title and price
                updated_product_title = driver.find_element(By.ID, 'productTitle').text
                try:
                    price = driver.find_element(By.XPATH, "//div[@id='corePriceDisplay_desktop_feature_div']//span[@class='a-price-whole']").text
                except:
                    price = "Price not available"

                combination = ", ".join([f"{variation_names[i]}: {selection.text if isinstance(selection, webdriver.remote.webelement.WebElement) else selection[1]}" for i, selection in enumerate(selections)])
                print(f"Selected Combination: {combination}")
                print(f"Updated Product Title: {updated_product_title}")
                print(f"Price: ₹{price}\n")

                return

            for option in variation_options[index]:
                if isinstance(option, webdriver.remote.webelement.WebElement):
                    iterate_variations(index + 1, selections + [option])
                else:
                    iterate_variations(index + 1, selections + [(dropdowns[0], option.text)])

        # Start the recursive iteration with the first variation
        iterate_variations(0, [])

    except Exception as e:
        print(f"An error occurred while fetching product variations: {e}")

    finally:
        driver.quit()

if __name__ == "__main__":
    # product_url = "https://www.amazon.in/dp/B0CP43RQY7"  #for 3 variations
    product_url = "https://www.amazon.in/dp/B09SHWH789"  #for 2 variations
    # product_url = "https://www.amazon.in/dp/B0CTKFW8FG"  #for 1 variations
    driver = initialize_driver()
    fetch_product_variations(driver, product_url)


There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache
Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)


WebDriverException: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=130.0.6723.117)
Stacktrace:
	GetHandleVerifier [0x00007FF70FB738A5+3004357]
	(No symbol) [0x00007FF70F809970]
	(No symbol) [0x00007FF70F6B582A]
	(No symbol) [0x00007FF70F6B2F26]
	(No symbol) [0x00007FF70F6A3E79]
	(No symbol) [0x00007FF70F6A5C16]
	(No symbol) [0x00007FF70F6A413F]
	(No symbol) [0x00007FF70F6A39CB]
	(No symbol) [0x00007FF70F6A390A]
	(No symbol) [0x00007FF70F6A153E]
	(No symbol) [0x00007FF70F6A1E0C]
	(No symbol) [0x00007FF70F6B88CA]
	(No symbol) [0x00007FF70F74C86E]
	(No symbol) [0x00007FF70F72BBDA]
	(No symbol) [0x00007FF70F74BA4C]
	(No symbol) [0x00007FF70F72B983]
	(No symbol) [0x00007FF70F6F7628]
	(No symbol) [0x00007FF70F6F8791]
	GetHandleVerifier [0x00007FF70FB9A00D+3161901]
	GetHandleVerifier [0x00007FF70FBEE060+3506048]
	GetHandleVerifier [0x00007FF70FBE400D+3465005]
	GetHandleVerifier [0x00007FF70F960EEB+830987]
	(No symbol) [0x00007FF70F81467F]
	(No symbol) [0x00007F